<a href="https://colab.research.google.com/github/AJ-Horch/credit_reating/blob/main/cleaning_credit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.5 MB/s eta 0:00:00


In [2]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, \
f1_score, roc_auc_score, roc_curve, auc, confusion_matrix, classification_report
import string
from sklearn.model_selection import train_test_split
import os
import statistics
import shutil
import matplotlib
import re
import warnings
import catboost
from catboost import CatBoostClassifier, cv
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
np.printoptions(pricision=2)
import pandas as pd
import itertools
pd.set_option('display.max_columns', 30)
# %matplotlib inline
warnings.filterwarnings('ignore')
# nltk.download('stopwords')
plt.style.use('ggplot')
from scipy.stats.contingency import chi2_contingency
from sklearn.feature_selection import chi2, SelectKBest, f_classif
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import boxcox, skew, kurtosis, normaltest
from sklearn.decomposition import PCA
import sklearn.impute
from catboost import Pool
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!pip install opendatasets
import opendatasets as od

user: ajhorch key: 7c9e1d1e4c9c2a5d7c3ad96d00a0b742

In [5]:
od.download('https://www.kaggle.com/datasets/parisrohan/credit-score-classification?select=test.csv')
od.download('https://www.kaggle.com/datasets/parisrohan/credit-score-classification?select=train.csv')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ajhorch
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/parisrohan/credit-score-classification


100%|██████████| 9.51M/9.51M [00:00<00:00, 11.9MB/s]



Skipping, found downloaded files in "./credit-score-classification" (use force=True to force download)


In [6]:
dtypes = dict(
    Month="category",
    Name="category",
    Occupation="category",
    Type_of_Loan="category",
    Credit_History_Age="category",
    Payment_Behaviour="category"
)


train_df = pd.read_csv("/content/credit-score-classification/train.csv", dtype=dtypes, parse_dates=['Month'])
train_df["is_train"] = True
test_df = pd.read_csv("//content/credit-score-classification/test.csv", dtype=dtypes, parse_dates=['Month'])
test_df["is_train"] = False
df = pd.concat([train_df, test_df])

In [7]:
df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,is_train
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good,True
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good,True
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good,True
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good,True
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good,True


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150000 entries, 0 to 49999
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   ID                        150000 non-null  object  
 1   Customer_ID               150000 non-null  object  
 2   Month                     150000 non-null  object  
 3   Name                      135000 non-null  category
 4   Age                       150000 non-null  object  
 5   SSN                       150000 non-null  object  
 6   Occupation                150000 non-null  category
 7   Annual_Income             150000 non-null  object  
 8   Monthly_Inhand_Salary     127500 non-null  float64 
 9   Num_Bank_Accounts         150000 non-null  int64   
 10  Num_Credit_Card           150000 non-null  int64   
 11  Interest_Rate             150000 non-null  int64   
 12  Num_of_Loan               150000 non-null  object  
 13  Type_of_Loan              132888 no

In [9]:
# what are the unique values in the Credit_Mix col?
np.unique(df["Credit_Mix"])

array(['Bad', 'Good', 'Standard', '_'], dtype=object)

In [10]:
# Drop irrelevant cols
df.drop(["Name", "SSD", "ID"], axis=1, inplace=True, errors="ignore")

### Looking fir unique values in categorical columns

This code extracts and analyzes categorical columns in a DataFrame, recording statistics about their unique values, counts, and percentages of occurance and returns this inofrmation in a structed DataFrame.

In [18]:
def get_unique_values(df):
  cat_cols = df.select_dtypes("object").columns

  data_info = np.zeros((len(cat_cols), 5), dtype='object')
  for i, col in enumerate(cat_cols):
    if len(df[col].unique()) > 5000:
      continue
    else:
      unique_values, counts = np.unique(
          np.array(df[col], dtype=str), return_counts=True)
      num_of_uv = len(unique_values)
      unique_val_percent = np.round(counts / counts.sum(), 2)
      data_info[i, :] = [col, unique_values.tolist(), counts.tolist, num_of_uv, unique_val_percent]
  return pd.DataFrame(data_info, columns=['column', 'unique', 'counts', 'len_unique_values',
                                            '%_unique_values'])

In [19]:
unique_values_df = get_unique_values(df)
unique_values_df.head()

,column,unique,counts,len_unique_values,%_unique_values
0,0,0,0,0,0
1,Month,"[April, August, December, February, January, J...",<built-in method tolist of numpy.ndarray objec...,12,"[0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.0..."
2,Age,"[-500, 100, 1004, 1006, 1007, 1010, 1018_, 102...",<built-in method tolist of numpy.ndarray objec...,2524,"[0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,0,0,0,0,0
4,0,0,0,0,0


### Data Processin with the DataProcessor Class

#### What is a Data Class?